In [1]:
import sys, os
sys.path.append(os.path.realpath(".."))
os.environ["TF_XLA_FLAGS"]="--tf_xla_cpu_global_jit"

import util_funcs
from importlib import reload
import data_reader as read
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import constants
import clinical_text_analysis as cta
import tsfresh
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from os import path
import keras_models.dataGen as dg

Using Theano backend.


In [2]:
import keras_models.vanPutten as vp

In [ ]:
query = {""}

In [16]:
reload(vp)
model = vp.inception_like((21,500,1), num_layers=9, num_filters=10)

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 21, 500, 1)   0                                            
__________________________________________________________________________________________________
conv2d_245 (Conv2D)             (None, 20, 499, 10)  50          input_10[0][0]                   
__________________________________________________________________________________________________
conv2d_254 (Conv2D)             (None, 19, 498, 10)  100         input_10[0][0]                   
__________________________________________________________________________________________________
conv2d_263 (Conv2D)             (None, 18, 497, 10)  170         input_10[0][0]                   
__________________________________________________________________________________________________
conv2d_272